In [19]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

In [20]:
data = pd.read_csv("../datos/Datos_12h_mean.csv")
data=data.iloc[:,1:]
data.head()

,id_batch,tiempo_acumulado,tiempo_final,CO2air__mean,Cum_irr__mean,EC_drain_PC__mean,HumDef__mean,PipeGrow__mean,PipeLow__mean,Rhair__mean,Tair__mean,Tot_PAR__mean,pH_drain_PC__mean
0,0,0.5,44.5,416.534722,1.265278,6.568056,1.234722,13.909722,41.995139,94.440972,24.093056,208.277778,6.048611
1,0,1.0,44.0,517.916667,4.353472,6.334722,2.244444,9.456944,36.657639,89.091667,22.234722,176.312500,6.017361
2,0,1.5,43.5,633.326389,0.750000,8.116667,2.258333,0.000000,38.254861,89.425000,23.465278,212.958333,7.288889
3,0,2.0,43.0,586.972222,2.025000,8.525000,3.450000,0.000000,36.361806,83.614583,22.282639,134.888889,7.324306
4,0,2.5,42.5,629.736111,0.947222,8.691667,2.585417,0.000000,30.606250,87.993750,23.677778,201.944444,7.306250


In [21]:
import numpy as np
import pandas as pd

# Columnas continuas
cols_continuas=['CO2air__mean', 'Cum_irr__mean', 'EC_drain_PC__mean', 'HumDef__mean',
 'PipeGrow__mean', 'PipeLow__mean', 'Rhair__mean', 'Tair__mean',
 'Tot_PAR__mean', 'pH_drain_PC__mean']


id = data["id_batch"]
lags = 10

lag_cols = []
for p in range(1, lags):
    for c in cols_continuas:
        col_name = f"{c}-{p}"  # nuevo nombre de columna
        data[col_name] = data[c].shift(p)  # shift de una sola columna
        data[col_name].fillna(data[col_name].mean(), inplace=True)  # rellenar NaN con la media
        lag_cols.append(col_name)

cols_continuas = cols_continuas + lag_cols
# Ver resultado
data[[c for c in data.columns if '__mean' in c]].head(10)

cols_continuas
X = data[cols_continuas].values
Y = data['tiempo_final'].values

/tmp/ipykernel_6598/2339927310.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col_name].fillna(data[col_name].mean(), inplace=True)  # rellenar NaN con la media
/tmp/ipykernel_6598/2339927310.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'd

In [ ]:
import numpy as np


def nw_class_prob_vectorized(X, Y, x, h=1.0):
    X = np.asarray(X)
    Y = np.asarray(Y)
    d = X.shape[1]
    
    # Covarianza escalada
    Sigma = np.cov(X, rowvar=False) * h**2 + 1e-6*np.eye(d)
    inv_Sigma = np.linalg.inv(Sigma)
    det_Sigma = np.linalg.det(Sigma)
    
    # Diferencias: cada fila de X menos x
    diffs = X - x  # shape (n_samples, d)
    
    # Distancia de Mahalanobis para todas las filas a la vez
    # (dif_i @ inv_Sigma @ dif_i.T) para cada i
    mdist2 = np.einsum('ij,jk,ik->i', diffs, inv_Sigma, diffs)
    
    # Pesos kernel
    weights = np.exp(-0.5 * mdist2) / np.sqrt((2*np.pi)**d * det_Sigma)
    
    weighted_prob = np.sum(weights*Y) /np.sum(weights)

    # Probabilidad de clase c
    return weighted_prob





In [ ]:
days_hat = []
for i in range(data.shape[0]):
    days_hat.append(nw_class_prob_vectorized(X,Y,data.loc[i,cols_continuas].values))

from sklearn.metrics import mean_squared_error,r2_score

mean_squared_error(y_true= Y, y_pred=days_hat)

20.974738060200842

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
r2_score(y_true= Y, y_pred=days_hat)

0.8876727820023019

array([[4.16534722e+02, 1.26527778e+00, 6.56805556e+00, ...,
        2.29983701e+01, 2.33244556e+02, 5.37675060e+00],
       [5.17916667e+02, 4.35347222e+00, 6.33472222e+00, ...,
        2.29983701e+01, 2.33244556e+02, 5.37675060e+00],
       [6.33326389e+02, 7.50000000e-01, 8.11666667e+00, ...,
        2.29983701e+01, 2.33244556e+02, 5.37675060e+00],
       ...,
       [6.59631944e+02, 5.26527778e+00, 7.06250000e+00, ...,
        2.19333333e+01, 1.88409722e+02, 4.15486111e+00],
       [7.63722222e+02, 7.09027778e-01, 7.25000000e+00, ...,
        2.40125000e+01, 3.99194444e+02, 4.18333333e+00],
       [7.07465278e+02, 4.27013889e+00, 6.78402778e+00, ...,
        2.17722222e+01, 1.80236111e+02, 4.15000000e+00]])